In [ ]:
import wandb
wandb.login()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision import models
import copy

In [ ]:
# Resnet18
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

In [ ]:
def freeze_layers(model, freeze_amt):
    model_copy = copy.deepcopy(model)
    
    params = list(model_copy.parameters())
    params_len = len(params)
    
    for child in params[:int(freeze_amt*params_len)]:
        child.requires_grad = False
    
    return model_copy

In [ ]:
# CIFAR10 dataset

# Transforms
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

# Dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, transform = transform, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform = transform)

# Train-val split
torch.manual_seed(39)
val_size = int(0.1*len(train_dataset))
train_size = len(train_dataset) - val_size
train, val = torch.utils.data.random_split(train_dataset, [train_size, val_size])

In [ ]:
# Dataloader & Hyperparameters
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
batch_size = 64
learning_rate = 1e-3
epochs = 10


train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val, batch_size=batch_size, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers=4)

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [ ]:
def train(train_loader, val_loader, model, amount, device=device):
    
  wandb.init(
    project = 'VCL_Tasks',
    group = 'Task_2',
    job_type = f'{amount*100}% Freeze',
  )
  
  model = freeze_layers(model, amount)
  model = model.to(device)
  criterion = nn.CrossEntropyLoss().to(device=device)
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)

  for epoch in range(epochs):
    # Training
    avg_train_loss = 0
    train_acc = []
    model.train() 
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(device=device)
        label = label.to(device=device)
        
        out = model(image)
        loss = criterion(out, label)
        acc = accuracy(out, label)
        train_acc.append(acc)
        avg_train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    avg_train_loss = avg_train_loss/len(train_loader)
    avg_train_acc = torch.stack(train_acc).mean()
    wandb.log({'Task_2/Train Accuracy' : avg_train_acc, 'Task_2/epoch':epoch+1})
    wandb.log({'Task_2/Train Loss':avg_train_loss, 'Task_2/epoch':epoch+1})


    # Validation 
    avg_val_loss = 0
    val_acc = []
    model.eval()
    for batch_idx, (image, label) in enumerate(val_loader):
        with torch.no_grad():
            image = image.to(device=device)
            label = label.to(device=device)

            out = model(image)
            loss = criterion(out, label)
            acc = accuracy(out, label)
            val_acc.append(acc)

            avg_val_loss += loss.item()
        
    avg_val_loss = avg_val_loss/len(val_loader)
    avg_val_acc = torch.stack(val_acc).mean()
    wandb.log({'Task_2/Validation Accuracy' : avg_val_acc, 'Task_2/epoch':epoch+1})
    wandb.log({'Task_2/Validation Loss' : avg_val_loss, 'Task_2/epoch':epoch+1})

    print(f'Epochs:{epoch+1}, Average Train Loss:{avg_train_loss}, Average Train Accuracy:{avg_train_acc}, Average Validation Loss:{avg_val_loss}, Validation Accuracy:{avg_val_acc}')

    torch.save(model.state_dict(), f'save_model/Task_2/model_finetune_freeze_{amount*100}.pth')
    wandb.save(f'save_model/Task_2/model_finetune_freeze_{amount*100}.pth')
  wandb.finish()  

In [ ]:
for amount in [0.1, 0.2 , 0.3]:
    train(train_loader, val_loader, model, amount, device=device)